In [45]:
import pandas as pd
import sqlite3

con = sqlite3.connect("data/nba.sqlite")
games = pd.read_sql_query("SELECT * FROM game", con)
active_players = pd.read_sql_query("SELECT full_name FROM player WHERE is_active=TRUE", con)



#QUESTION 1: WHICH TEAMS IN THE PAST 5 YEARS CONSISTENTLY SCORE THE HIGHEST IN THEIR GAMES? 

games_past_5yrs = pd.read_sql_query("""
	SELECT * FROM game  
	WHERE game_date BETWEEN '2019-01-01 00:00:00' AND '2023-12-31 23:59:59'
    ORDER BY game_date DESC
""", con)

# Calculate statistics for home and away teams
home_stats = games_past_5yrs.groupby(['team_id_home', 'team_name_home'])['pts_home'].agg(['mean', 'std', 'count'])
away_stats = games_past_5yrs.groupby(['team_id_away', 'team_name_away'])['pts_away'].agg(['mean', 'std', 'count'])

home_stats.index.names = ['team_id', 'team_name']
away_stats.index.names = ['team_id', 'team_name']

combined_stats = pd.concat([home_stats, away_stats])
score_stats_all = combined_stats.groupby(['team_id', 'team_name']).mean()
score_stats_all['total_games'] = combined_stats.groupby(['team_id', 'team_name'])['count'].sum()

# Filter teams with at least 100 games
score_stats_all = score_stats_all[score_stats_all['total_games'] >= 100]

# Display top 25 teams by average score
print("Teams with highest average scores:")
print(score_stats_all.sort_values(by='mean', ascending=False).head(25))



#QUESTION 2: WHICH TEAMS SCORED BELOW AVERAGE THE MOST CONSECUTIVELY? 

# Calculate the average points per season
season_avg = pd.concat([games_past_5yrs['pts_home'], games_past_5yrs['pts_away']]).mean()

# Prepare home and away game data
home_games = games_past_5yrs[['season_id', 'team_id_home', 'team_name_home', 'pts_home']].rename(columns={
    'team_id_home': 'team_id',
    'team_name_home': 'team_name',
    'pts_home': 'points'
})
away_games = games_past_5yrs[['season_id', 'team_id_away', 'team_name_away', 'pts_away']].rename(columns={
    'team_id_away': 'team_id',
    'team_name_away': 'team_name',
    'pts_away': 'points'
})

# Combine home and away games into a single dataset
all_games = pd.concat([home_games, away_games])

# Calculate average points per team per season
team_season_avg = all_games.groupby(['season_id', 'team_id', 'team_name'])['points'].mean().reset_index()

# Determine below-average seasons
team_season_avg['league_avg'] = season_avg
team_season_avg['below_average'] = team_season_avg['points'] < team_season_avg['league_avg']
team_season_avg = team_season_avg.sort_values(['team_id', 'season_id'])

# Display the first 10 records
print("Teams who scored below average for the most consecutive seasons")
print(team_season_avg.head(10))



#QUESTION 3: WHICH TEAMS HAVE HAD THE MOST CONSISTENT APPEARENCES IN THE PLAYOFFS?

# Analyze consistent playoff appearances
playoff_games = games_recent_5yrs[games_recent_5yrs['season_type'] == 'Playoffs']

# Get distinct playoff appearances per team per season
playoff_home_seasons = playoff_games[['season_id', 'team_name_home']].rename(columns={'team_name_home': 'team_name'}).drop_duplicates()
playoff_away_seasons = playoff_games[['season_id', 'team_name_away']].rename(columns={'team_name_away': 'team_name'}).drop_duplicates()
all_playoff_seasons = pd.concat([playoff_home_seasons, playoff_away_seasons]).drop_duplicates()

# Count playoff appearances by team
playoff_appearances = all_playoff_seasons.groupby('team_name').size().sort_values(ascending=False)

# Create a dataframe for playoff appearances
consistent_playoff_teams = playoff_appearances.reset_index()
consistent_playoff_teams.columns = ['team_name', 'appearances']

# Sort teams by the most consistent playoff appearances
consistent_playoff_teams = consistent_playoff_teams.sort_values(by='appearances', ascending=False)

# Display results
print("\nTeams with the most consistent playoff appearances (2019-2023):")
print(consistent_playoff_teams)



#QUESTION 4: WHICH TEAMS HAVE IMPROVED THE MOST IN THE LAST 5 YEARS?

# Extract year and filter for regular season games
games_recent_5yrs['game_year'] = pd.to_datetime(games_recent_5yrs['game_date']).dt.year
regular_games = games_recent_5yrs[games_recent_5yrs['season_type'] == 'Regular Season']

# Combine home and away data for regular games
reg_all_games = pd.concat([
    regular_games[['team_name_home', 'game_year', 'pts_home']].rename(columns={'team_name_home': 'team_name', 'pts_home': 'points'}),
    regular_games[['team_name_away', 'game_year', 'pts_away']].rename(columns={'team_name_away': 'team_name', 'pts_away': 'points'})
])

# Calculate average points by team and year
team_year_avg = reg_all_games.groupby(['team_name', 'game_year'])['points'].mean().reset_index()

# Calculate total improvement for each team
team_improvement = team_year_avg.groupby('team_name')['points'].apply(lambda x: x.diff().sum()).reset_index()
team_improvement.columns = ['team_name', 'total_improvement']
team_improvement = team_improvement.sort_values(by='total_improvement', ascending=False)

# Display top teams by improvement
print("Teams with the greatest improvement over the past five years:")
print(team_improvement.head(20))



#QUESTION 5: WHICH TEAMS HAVE THE BEST DEFENSE? (BLOCKS/STEALS)

# Extract and combine defensive stats from home and away games
defensive_stats = pd.concat([
    regular_games[['team_name_home', 'blk_home', 'stl_home', 'wl_home']].rename(
        columns={'team_name_home': 'team_name', 'blk_home': 'blocks', 'stl_home': 'steals', 'wl_home': 'win_loss'}),
    regular_games[['team_name_away', 'blk_away', 'stl_away', 'wl_away']].rename(
        columns={'team_name_away': 'team_name', 'blk_away': 'blocks', 'stl_away': 'steals', 'wl_away': 'win_loss'})
])

# Group by team and calculate defensive performance metrics
team_defense = defensive_stats.groupby('team_name').agg(
    total_blocks=('blocks', 'sum'),
    total_steals=('steals', 'sum'),
    win_rate=('win_loss', lambda x: (x == 'W').mean())
).reset_index()

# Compute defensive effectiveness
team_defense['defensive_effectiveness'] = (team_defense['total_blocks'] + team_defense['total_steals']) / team_defense['win_rate']

# Sort teams by defensive effectiveness in descending order
team_defense_sorted = team_defense.sort_values(by=['defensive_effectiveness', 'win_rate'], ascending= [False, False])

# Display teams by defensive effectiveness in order from best to worst
print("Teams Ranked by Defensive Effectiveness (Best to Worst):")
print(team_defense_sorted)



Teams with highest average scores:
                                         mean        std  count  total_games
team_id    team_name                                                        
1610612749 Milwaukee Bucks         116.435716  12.726194  217.0          434
1610612762 Utah Jazz               114.124126  11.353901  196.5          393
1610612737 Atlanta Hawks           114.082051  12.425118  195.0          390
1610612750 Minnesota Timberwolves  114.046378  11.995903  184.0          368
1610612758 Sacramento Kings        113.760321  12.404773  186.5          373
1610612744 Golden State Warriors   113.553368  12.502250  207.5          415
1610612751 Brooklyn Nets           113.466575  12.269783  195.5          391
1610612740 New Orleans Pelicans    113.429579  11.357125  184.5          369
1610612763 Memphis Grizzlies       113.219401  12.564816  196.5          393
1610612756 Phoenix Suns            113.174049  11.591735  205.5          411
1610612743 Denver Nuggets          112.98